In [1]:
!pip3 install git+https://github.com/JustAnotherArchivist/snscrape.git
!pip install pandas

  Cloning https://github.com/JustAnotherArchivist/snscrape.git to /tmp/pip-req-build-89jsmwuz
  Running command git clone --filter=blob:none --quiet https://github.com/JustAnotherArchivist/snscrape.git /tmp/pip-req-build-89jsmwuz
  Resolved https://github.com/JustAnotherArchivist/snscrape.git to commit 614d4c2029a62d348ca56598f87c425966aaec66
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for snscrape: filename=snscrape-0.7.0.20230622-py3-none-any.whl size=74814 sha256=f7e1468e95dae85f223d4c833086170c8779e29d857047edac19618d82f92ac8
  Stored in directory: /tmp/pip-ephem-wheel-cache-0q2ybs6e/wheels/05/e9/f7/57056e7c7e44b1feed932fa49fdec9d706c4f563e37160ab74
Successfully built snscrape


In [7]:
import os
import datetime

# Batasi jumlah hasil yang diambil
max_results = 10000

# Gunakan Twitter search untuk mencari tweet yang di-favoritkan minimal 5000 kali dan berbahasa Indonesia
twitter_search = "sektor saham lang:id since:2018-06-29 until:2023-06-29"

# Tentukan nama file dengan format "<kueri pencarian>_<tanggal saat ini>.json"
filename = f"{twitter_search.replace(' ', '_').replace(':', '-').replace('#', '')}_{datetime.date.today().strftime('%Y-%m-%d')}.json"

USING_TOP_SEARCH = False

snscrape_params = '--jsonl --max-results'
twitter_search_params = ''

if USING_TOP_SEARCH:
    twitter_search_params += "--top"

snscrape_search_query = f"snscrape {snscrape_params} {max_results} twitter-search {twitter_search_params} '{twitter_search}' > {filename}"

print(snscrape_search_query)

os.system(snscrape_search_query)

snscrape --jsonl --max-results 10000 twitter-search  'sektor saham lang:id since:2018-06-29 until:2023-06-29' > sektor_saham_lang-id_since-2018-06-29_until-2023-06-29_2023-06-29.json


0

In [10]:
import pandas as pd
import ast
import json

# Membaca file JSON hasil dari perintah CLI sebelumnya dan membuat dataframe pandas
tweets_df = pd.read_json(filename, lines=True)

NAMA_FILE_CSV = 'sahamsektor.csv'

# Membuat kamus untuk mengganti nama kolom
new_columns = {
    'conversationId': 'Conv. ID',
    'url': 'URL',
    'date': 'Date',
    'rawContent': 'Tweet',
    'id': 'ID',
    'replyCount': 'Replies',
    'retweetCount': 'Retweets',
    'likeCount': 'Likes',
    'quoteCount': 'Quotes',
    'bookmarkCount': 'Bookmarks',
    'lang': 'Language',
    'links': 'Links',
    'media': 'Media',
    'retweetedTweet': 'Retweeted Tweet',
    'username': 'Username'
}

if len(tweets_df) == 0:
    print('Pencarian tidak ditemukan coba ganti keyword lain, keywordmu: ', twitter_search)
    exit()
else:
  # Memilih kolom yang akan digunakan dan mengganti nama kolom menggunakan kamus yang telah dibuat
  tweets_df = tweets_df.loc[:, ['url', 'date', 'rawContent', 'id',
                            'replyCount', 'retweetCount', 'likeCount', 'quoteCount',
                            'conversationId', 'lang', 'links',
                            'media', 'retweetedTweet', 'bookmarkCount', 'username']]
  tweets_df = tweets_df.rename(columns=new_columns)

  # Ekstrak fullUrl dari kolom media dan url dari kolom links
  tweets_df['Media'] = tweets_df['Media'].apply(lambda x: x[0]['fullUrl'] if isinstance(x, list) and x and isinstance(x[0], dict) and 'fullUrl' in x[0] else None)
  tweets_df['Links'] = tweets_df['Links'].apply(lambda x: x[0]['url'] if isinstance(x, list) and x and isinstance(x[0], dict) and 'url' in x[0] else None)

  # Menampilkan dataframe tweets_df
  display(tweets_df)

  # Simpan ke csv
  tweets_df.to_csv(NAMA_FILE_CSV, index=False)


,URL,Date,Tweet,ID,Replies,Retweets,Likes,Quotes,Conv. ID,Language,Links,Media,Retweeted Tweet,Bookmarks,Username
0,https://twitter.com/azmi_radarsaham/status/167...,2023-06-28 06:40:22+00:00,Data Sektor Perladangan &amp; saham pilihan pe...,1673944409875042304,0,1,22,0,1673944409875042304,in,None,https://pbs.twimg.com/media/FzsLjrMaMAAR8Lp?fo...,NaN,1,azmi_radarsaham
1,https://twitter.com/merakitt/status/1673858120...,2023-06-28 00:57:29+00:00,"Rill, saham sektor perbankan di Indo bisa dape...",1673858120886652928,0,0,0,0,1673858120886652928,in,None,None,NaN,0,merakitt
2,https://twitter.com/SekuritasBahana/status/167...,2023-06-27 13:15:01+00:00,Indeks Harga Saham Gabungan (IHSG) ditutup mel...,1673681340716154880,0,0,1,0,1673681340716154880,in,None,https://pbs.twimg.com/media/FzocTifakAYIToB?fo...,NaN,0,SekuritasBahana
3,https://twitter.com/MaybankTradeID/status/1673...,2023-06-27 10:26:55+00:00,Dear MayFriends\n\nIndeks Harga Saham Gabungan...,1673639036681068545,1,0,0,0,1673639036681068545,in,None,None,NaN,0,MaybankTradeID
4,https://twitter.com/petrikbintanggg/status/167...,2023-06-27 08:58:04+00:00,@Polentacheese @zanuarherrera20 @utdfocusid @M...,1673616674833195009,1,0,0,0,1673550539043520512,in,None,None,NaN,0,petrikbintanggg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2500,https://twitter.com/Nandda31/status/1530119812...,2022-05-27 09:32:46+00:00,Kepala Ekonom PT Bank Central Asia Tbk David E...,1530119812252602368,0,0,0,0,1530119812252602368,in,https://www.antaranews.com/berita/2903277/ekon...,None,NaN,0,Nandda31
2501,https://twitter.com/andkfyh/status/15301191223...,2022-05-27 09:30:01+00:00,saham emiten sektor teknologi saat ini kembali...,1530119122348122118,0,0,0,0,1530119122348122118,in,https://www.antaranews.com/berita/2903277/ekon...,None,NaN,0,andkfyh
2502,https://twitter.com/Sukinem7979/status/1530119...,2022-05-27 09:30:01+00:00,"Setelah mengalami tekanan, saham emiten sektor...",1530119120217325568,0,0,0,0,1530119120217325568,in,https://www.cnbcindonesia.com/market/202205261...,None,NaN,0,Sukinem7979
2503,https://twitter.com/Guranma11/status/153011911...,2022-05-27 09:30:00+00:00,"Setelah mengalami tekanan, saham emiten sektor...",1530119119499980801,0,0,0,0,1530119119499980801,in,https://m.tribunnews.com/bisnis/2022/05/26/sah...,None,NaN,0,Guranma11


In [11]:
# Cek jumlah data yang didapatkan

num_tweets = len(tweets_df)
print(f"Jumlah tweet dalam dataframe adalah {num_tweets}.")

Jumlah tweet dalam dataframe adalah 2505.
